In [1]:
import pandas as pd
import numpy as np

In [2]:
file = pd.read_csv('../csvs/species_height_nitesh.csv')

In [ ]:
# df.rename(columns={'species_na':'species_name','Elevation_dem':'altitude','exif_byte_':'exif_byte_order',
#      'phone_came':'phone_camera_model','encoding_p':'encoding_process', 'rear_main_':'rear_main_camera_megapixels',
#        'rear_main1':'rear_main_camera_aperture_size','rear_camer':'rear_camera_flash_available',
#         'original_i':'original_image_size', 'original_1':'original_image_resolution',  'image_date':'image_date_time'                                            
#        }, inplace=True)

In [49]:
desired_columns = ['Elevation_dem', 'altitude','latitude','longitude','accuracy','species_na', 'state_name',]

In [50]:
file = file[desired_columns]

In [51]:
file.rename(columns={
    'species_na':'species_name','Elevation_dem':'altitude_dem'
},inplace=True)

In [53]:
file.to_csv("../csvs/metadata_v1.csv",index=False)

In [39]:
file.columns

Index(['Elevation_dem', 'altitude', 'time_of_da', 'phone_came', 'latitude',
       'rear_camer', 'species_na', 'accuracy', 'rear_main1', 'image_date',
       'original_i', 'encoding_p', 'original_1', 'month', 'state_name',
       'exif_byte_', 'phone_make', 'processed_', 'image_id', 'rear_main_',
       'system:index', 'longitude'],
      dtype='object')

In [40]:
file['diff'] = file['Elevation_dem'].astype(int)-file['altitude'].astype(int)

In [45]:
disered_cols = ['Elevation_dem','altitude','diff']
temp = file[disered_cols]
temp.to_csv('./temp.csv',index=False)

In [46]:
temp['diff'].describe()

count    158586.000000
mean         44.036271
std          90.591631
min       -5405.000000
25%          22.000000
50%          38.000000
75%          54.000000
max        2580.000000
Name: diff, dtype: float64

In [ ]:
# species_bounds = {
#     # "Aesculus indica": (1500, 2800), 1000 - 2700
#     # "Buchanania lanzan": (40, 700),   1000
#     # "Cedrus deodara": (1000, 3000), 1200 - 3200
#     "Eucalyptus globulus": (300, 1800), 
#     # "Madhuca longifolia": (0, 600), 1200
#     # "Mangifera sylvatica": (300, 1200), 1500
#     # "Phyllanthus emblica": (300, 1200), 1800
#     # "Pinus roxburghii": (300, 2200), 4000
#     "Quercus leucotrichophora": (900, 2600),
#     # "Rhododendron arboreum": (1300, 3000), 1000 - 3800
#     # "Senegalia catechu": (200, 1000), 2000
#     # "Shorea robusta": (100, 1200),  1700
#     "Taxus baccata": (1000, 3100),
# }

In [ ]:
species_bounds = {
    "Aesculus indica": (1000, 2700),
    "Buchanania lanzan": (40, 1000),   
    "Cedrus deodara": (1200, 3200),
    "Eucalyptus globulus": (300, 1800), 
    "Madhuca longifolia": (0,  1200),
    "Mangifera sylvatica": (300, 1500), 
    "Phyllanthus emblica": (300, 1800),
    "Pinus roxburghii": (300, 4000), 
    "Quercus leucotrichophora": (900, 2600),
    "Rhododendron arboreum": (1000, 3800),
    "Senegalia catechu": (200, 2000),
    "Shorea robusta": (100, 2000),
    "Taxus baccata": (1000, 3100),
}

In [14]:
def is_outlier(row):
    min_elev, max_elev = species_bounds.get(row['species_na'], (None, None))
    if min_elev is None or max_elev is None:
        return True   
    return not (min_elev <= row['Elevation_dem'] <= max_elev)


In [15]:
file['outlier'] = file.apply(is_outlier, axis=1)

In [16]:
file['species_na'].value_counts()

species_na
Pinus roxburghii            41163
Shorea robusta              24970
Senegalia catechu           23341
Cedrus deodara              21201
Eucalyptus globulus          8443
Quercus leucotrichophora     6495
Phyllanthus emblica          6453
Madhuca longifolia           6084
Mangifera sylvatica          4855
Rhododendron arboreum        4738
Aesculus indica              4285
Buchanania lanzan            3363
Taxus baccata                3195
Name: count, dtype: int64

In [17]:
def assign_elevation_levels(group):
    non_outliers = group[~group['outlier']].copy()
    
    if len(non_outliers) < 5:
        group['elevation_level'] = np.nan
    else:
        try: 
            elevation_levels = pd.qcut(
                non_outliers['Elevation_dem'],
                q=5,
                labels=['level1', 'level2', 'level3', 'level4', 'level5']
            ) 
            group['elevation_level'] = np.nan
            group.loc[non_outliers.index, 'elevation_level'] = elevation_levels
        except ValueError:
            group['elevation_level'] = np.nan

    return group


In [18]:
df = file.groupby('species_na', group_keys=False).apply(assign_elevation_levels)

C:\Users\31733\AppData\Local\Temp\ipykernel_10736\2317437317.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['level4', 'level1', 'level1', 'level1', 'level1', ..., 'level3', 'level3', 'level3', 'level3', 'level3']
Length: 4234
Categories (5, object): ['level1' < 'level2' < 'level3' < 'level4' < 'level5']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  group.loc[non_outliers.index, 'elevation_level'] = elevation_levels
C:\Users\31733\AppData\Local\Temp\ipykernel_10736\2317437317.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['level2', 'level2', 'level1', 'level1', 'level1', ..., 'level4', 'level4', 'level4', 'level4', 'level4']
Length: 3363
Categories (5, object): ['level1' < 'level2' < 'level3' < 'level4' < 'level5']' has dtype incompatible with float64, please expli

In [19]:
df['elevation_level'].value_counts()

elevation_level
level1    31091
level3    30902
level2    30812
level5    30789
level4    30667
Name: count, dtype: int64

In [20]:
df.columns

Index(['Elevation_dem', 'altitude', 'time_of_da', 'phone_came', 'latitude',
       'rear_camer', 'species_na', 'accuracy', 'rear_main1', 'image_date',
       'original_i', 'encoding_p', 'original_1', 'month', 'state_name',
       'exif_byte_', 'phone_make', 'processed_', 'image_id', 'rear_main_',
       'system:index', 'longitude', 'outlier', 'elevation_level'],
      dtype='object')

In [21]:
df['month'].value_counts()

month
July         67103
May          30095
June         26249
August       19070
April         6886
October       3556
September     2834
November      2551
December       242
Name: count, dtype: int64

In [22]:
leaf_status_dict = {
    'Aesculus indica': {
        'leaf_on': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'leaf_off': [11, 12, 1]
    },
    'Buchanania lanzan': {
        'leaf_on': [2, 3, 4, 5, 6, 7, 8, 9, 10],
        'leaf_off': [11, 12, 1]
    },
    'Cedrus deodara': 'evergreen',
    'Eucalyptus globulus': 'evergreen',
    'Madhuca longifolia': {
        'leaf_on': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        'leaf_off': [12, 1]
    },
    'Mangifera sylvatica': {
        'leaf_on': [3, 4, 5, 6, 7, 8, 9, 10, 11],
        'leaf_off': [12, 1, 2]
    },
    'Phyllanthus emblica': {
        'leaf_on': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'leaf_off': [1, 2]
    },
    'Pinus roxburghii': 'evergreen',
    'Quercus leucotrichophora': 'evergreen',
    'Rhododendron arboreum': {
        'leaf_on': [1,2,3,4,5,6, 7, 8, 9,12],
        'leaf_off': [10, 11]
    },
    'Senegalia catechu': {
        'leaf_on': [1,5,6, 7,8,9,10,11,12],
        'leaf_off': [2, 3, 4]
    },
    'Shorea robusta': {
        'leaf_on': [1,5,6, 7,8,9,10,11,12],
        'leaf_off': [2, 3, 4]
    },
    'Taxus baccata': 'evergreen'
}

In [23]:
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}


In [24]:
df['month_num'] = df['month'].map(month_map)

In [25]:
def get_leaf_status(row):
    species = row['species_na']
    month = row['month_num']
    
    info = leaf_status_dict.get(species)
    
    if info == 'evergreen':
        return 'leaf_on'
    elif info:
        if month in info['leaf_on']:
            return 'leaf_on'
        elif month in info['leaf_off']:
            return 'leaf_off'
    return 'unknown'


In [26]:
df['leaf_status'] = df.apply(get_leaf_status, axis=1)


In [27]:
df['leaf_status'].value_counts()

leaf_status
leaf_on     152278
leaf_off      6308
Name: count, dtype: int64

In [28]:
df.columns

Index(['Elevation_dem', 'altitude', 'time_of_da', 'phone_came', 'latitude',
       'rear_camer', 'species_na', 'accuracy', 'rear_main1', 'image_date',
       'original_i', 'encoding_p', 'original_1', 'month', 'state_name',
       'exif_byte_', 'phone_make', 'processed_', 'image_id', 'rear_main_',
       'system:index', 'longitude', 'outlier', 'elevation_level', 'month_num',
       'leaf_status'],
      dtype='object')

In [29]:
desired_columns =[
    'image_id','species_na','latitude','longitude','Elevation_dem','state_name','exif_byte_','phone_make',
    'phone_came','encoding_p','rear_main_','rear_main1','rear_camer','original_i','original_1','image_date','outlier', 'elevation_level','leaf_status'
]

In [30]:
df = df[desired_columns]

In [31]:
df.rename(columns={'species_na':'species_name','Elevation_dem':'altitude','exif_byte_':'exif_byte_order',
     'phone_came':'phone_camera_model','encoding_p':'encoding_process', 'rear_main_':'rear_main_camera_megapixels',
       'rear_main1':'rear_main_camera_aperture_size','rear_camer':'rear_camera_flash_available',
        'original_i':'original_image_size', 'original_1':'original_image_resolution',  'image_date':'image_date_time'                                            
       }, inplace=True)

In [32]:
df['elevation_level'].fillna('level0',inplace=True)

C:\Users\31733\AppData\Local\Temp\ipykernel_10736\3054686369.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['elevation_level'].fillna('level0',inplace=True)


In [33]:
df.columns

Index(['image_id', 'species_name', 'latitude', 'longitude', 'altitude',
       'state_name', 'exif_byte_order', 'phone_make', 'phone_camera_model',
       'encoding_process', 'rear_main_camera_megapixels',
       'rear_main_camera_aperture_size', 'rear_camera_flash_available',
       'original_image_size', 'original_image_resolution', 'image_date_time',
       'outlier', 'elevation_level', 'leaf_status'],
      dtype='object')

In [36]:
df['outlier'].value_counts()

outlier
False    154261
True       4325
Name: count, dtype: int64

In [35]:
df.to_csv('../csvs/Updated_Metadata.csv',index=False)